# Testing Homography / Perspektiv Transformation

In [1]:
import cv2
import numpy as np

In [2]:
# Load the image and the floor plan
image = cv2.imread("Ch4_image.png")

In [3]:
def select_points(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        # Add the selected point to the list of points
        param[0].append([x,y])

def get_points(img):
    # Create a copy of the image for display purposes
    img_copy = img.copy()
    
    # List to store the selected points
    points = []
    
    # Set up the callback function for mouse events
    cv2.namedWindow("Image")
    cv2.setMouseCallback("Image", select_points, [points])
    
    # Loop until the user presses the 'q' key
    while True:
        # Show the image
        cv2.imshow("Image", img_copy)
        
        # Draw the selected points on the image
        for point in points:
            cv2.circle(img_copy, tuple(point), 5, (0,0,255), -1)
        
        # Check for user input
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break
    
    # Destroy the window
    cv2.destroyAllWindows()
    
    # Convert the list of points to a numpy array
    points = np.array(points, dtype=np.float32)
    
    # Compute the start and end points of the destination line
    start = points[0]
    end = points[-1]
    
    # Compute the equation of the line between the start and end points
    a = end[1] - start[1]
    b = start[0] - end[0]
    c = end[0]*start[1] - start[0]*end[1]
    
    # Compute the destination points for each source point
    dst_pts = np.zeros_like(points)
    for i, point in enumerate(points):
        x, y = point
        dst_pts[i,0] = (b*(b*x - a*y) - a*c)/(a**2 + b**2)
        dst_pts[i,1] = (a*(-b*x + a*y) - b*c)/(a**2 + b**2)
    
    # Return the source and destination points
    return points, dst_pts

def get_lines(image, n):
    src_pts = np.array([]).reshape(0,2)
    dst_pts = np.array([]).reshape(0,2)
    for i in range(n):
        src, dst = get_points(image)
        src_pts = np.concatenate((src_pts, src))
        dst_pts = np.concatenate((dst_pts, dst))
    return src_pts, dst_pts


In [4]:
src_pts, dst_pts = get_lines(image, 2)

In [5]:
len(src_pts), len(dst_pts)

(9, 9)

In [6]:
H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

In [7]:
H

array([[ 1.00635027e+00, -1.22039766e-03, -3.46788123e+00],
       [ 2.25888181e-04,  1.00291288e+00, -2.70978374e-01],
       [ 2.45734208e-06, -6.39581849e-07,  1.00000000e+00]])

In [8]:
image.shape

(1944, 2592, 3)

In [9]:
undistorted_img = cv2.warpPerspective(image, H, (2592, 1944))

In [10]:

# Save the undistorted image
cv2.imwrite("undistorted_image.jpg", undistorted_img)

True

## Calibarte Camera

In [12]:
camera_matrix = np.array([[1000, 0, 500],
                          [0, 1000, 500],
                          [0, 0, 1]])

distortion_coefficients = np.array([0.1, 0.1, 0, 0, 0])

# Correct the barrel distortion
undistorted_img = cv2.undistort(image, camera_matrix, distortion_coefficients)

# Save the undistorted image
cv2.imwrite('undistorted_matrix_image.jpg', undistorted_img)

True

: 